# SPORTS ANALYTICS

In [3]:
import pyodbc

SERVER = r'DESKTOP-7FS2HHR\SQLEXPRESS'
DATABASE = 'SportsAnalyticsDB'

CONN_STR = fr'DRIVER={{SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'

conn = pyodbc.connect(CONN_STR)
cursor = conn.cursor()

print("Connected successfully!")

Connected successfully!


In [ ]:
import pyodbc
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd

# SQL Server Connection
def connect_to_db():
    try:
        conn = pyodbc.connect(
            r'DRIVER={ODBC Driver 17 for SQL Server};'
            r'SERVER=DESKTOP-7FS2HHR\SQLEXPRESS;'
            r'DATABASE=SportsAnalyticsDB;'
            r'Trusted_Connection=yes;'
        )
        return conn
    except pyodbc.Error as e:
        messagebox.showerror("Database Connection Error", str(e))
        return None

QUERIES = {
    "1. Top 5 Highest Scoring Matches": """SELECT TOP 5 m.match_id, ht.team_name AS home_team, at.team_name AS away_team,
    m.home_score, m.away_score, (m.home_score + m.away_score) AS total_goals FROM Matches m
    JOIN Teams ht ON m.home_team_id = ht.team_id JOIN Teams at ON m.away_team_id = at.team_id ORDER BY total_goals DESC;""",

    "2. Top 10 Teams with Most Matches Played": """SELECT TOP 10 t.team_name, COUNT(*) AS matches_played FROM Teams t
    JOIN Matches m ON t.team_id = m.home_team_id OR t.team_id = m.away_team_id GROUP BY t.team_name ORDER BY matches_played DESC;""",

    "3. Top 10 Teams with Most Wins": """SELECT TOP 10 t.team_name, COUNT(*) AS wins FROM Teams t
    JOIN Matches m ON (t.team_id = m.home_team_id AND m.home_score > m.away_score)
    OR (t.team_id = m.away_team_id AND m.away_score > m.home_score)
    GROUP BY t.team_name ORDER BY wins DESC;""",

    "4. Top 5 Tournaments with Most Matches": """SELECT TOP 5 t.tournament_name, COUNT(m.match_id) AS matches_played
    FROM Tournaments t JOIN Matches m ON t.tournament_id = m.tournament_id GROUP BY t.tournament_name ORDER BY matches_played DESC;""",

    "5. Average Goals Per Match by Tournament": """SELECT t.tournament_name, 
    AVG(CAST(m.home_score + m.away_score AS FLOAT)) AS avg_goals_per_match 
    FROM Tournaments t JOIN Matches m ON t.tournament_id = m.tournament_id 
    GROUP BY t.tournament_name ORDER BY avg_goals_per_match DESC;""",

    "6. Top 10 Cities Hosting Most Matches": """SELECT TOP 10 city, COUNT(*) AS matches_hosted FROM Matches GROUP BY city ORDER BY matches_hosted DESC;""",

    "7. Top 10 Teams with Most Draws": """SELECT TOP 10 t.team_name, COUNT(*) AS draws FROM Teams t
    JOIN Matches m ON (t.team_id = m.home_team_id OR t.team_id = m.away_team_id)
    WHERE m.home_score = m.away_score GROUP BY t.team_name ORDER BY draws DESC;""",

    "8. Highest Scoring Match in Each Tournament": """WITH MatchGoals AS (
        SELECT m.match_id, m.tournament_id, (m.home_score + m.away_score) AS total_goals FROM Matches m)
        SELECT t.tournament_name, mg.match_id, mg.total_goals FROM MatchGoals mg
        JOIN Tournaments t ON mg.tournament_id = t.tournament_id
        WHERE mg.total_goals = (
            SELECT MAX(total_goals) FROM MatchGoals WHERE tournament_id = mg.tournament_id
        ) ORDER BY t.tournament_name;""",

    "9. Match Count by Year": """SELECT YEAR(match_date) AS match_year, COUNT(*) AS matches_played 
    FROM Matches GROUP BY YEAR(match_date) ORDER BY match_year;""",

    "10. Top 10 Teams with Highest Average Goals": """SELECT TOP 10 t.team_name, 
    CAST(SUM(CASE WHEN m.home_team_id = t.team_id THEN m.home_score 
                  WHEN m.away_team_id = t.team_id THEN m.away_score END) AS FLOAT) / COUNT(*) AS avg_goals_per_match 
    FROM Teams t JOIN Matches m ON t.team_id = m.home_team_id OR t.team_id = m.away_team_id 
    GROUP BY t.team_name ORDER BY avg_goals_per_match DESC;""",

    "11. Win Percentage of Each Team (Played Atleast 10 Matches)": """SELECT t.team_name, COUNT(*) AS matches_played,
    SUM(CASE WHEN t.team_id = m.home_team_id AND m.home_score > m.away_score THEN 1
             WHEN t.team_id = m.away_team_id AND m.away_score > m.home_score THEN 1 ELSE 0 END) AS wins,
    ROUND(CAST(SUM(CASE WHEN t.team_id = m.home_team_id AND m.home_score > m.away_score THEN 1
                        WHEN t.team_id = m.away_team_id AND m.away_score > m.home_score THEN 1 END) AS FLOAT)
                        * 100 / COUNT(*), 2) AS win_percentage
    FROM Teams t JOIN Matches m ON t.team_id = m.home_team_id OR t.team_id = m.away_team_id
    GROUP BY t.team_name HAVING COUNT(*) >= 10 ORDER BY win_percentage DESC;""",

    "12. Brazil vs Argentina Matches": """SELECT m.match_date, ht.team_name AS home_team, at.team_name AS away_team,
    m.home_score, m.away_score, t.tournament_name, m.city, m.country 
    FROM Matches m 
    JOIN Teams ht ON m.home_team_id = ht.team_id 
    JOIN Teams at ON m.away_team_id = at.team_id 
    JOIN Tournaments t ON m.tournament_id = t.tournament_id 
    WHERE (ht.team_name = 'Brazil' AND at.team_name = 'Argentina') 
       OR (ht.team_name = 'Argentina' AND at.team_name = 'Brazil') 
    ORDER BY m.match_date DESC;"""
}

def run_query():
    conn = connect_to_db()
    if conn:
        try:
            query_name = query_var.get()
            sql = QUERIES[query_name]
            df = pd.read_sql(sql, conn)
            for i in tree.get_children():
                tree.delete(i)
            tree['columns'] = list(df.columns)
            tree['show'] = 'headings'
            for col in df.columns:
                tree.heading(col, text=col)
                tree.column(col, anchor='center')
            for row in df.itertuples(index=False):
                tree.insert("", "end", values=row)
        except Exception as e:
            messagebox.showerror("Query Execution Error", str(e))
        finally:
            conn.close()

root = tk.Tk()
root.title("Sports Analytics")
root.geometry("1200x600")

query_var = tk.StringVar(value=list(QUERIES.keys())[0])

ttk.Label(root, text="Select Query").pack(pady=5)
query_menu = ttk.Combobox(root, textvariable=query_var, values=list(QUERIES.keys()), width=90, state="readonly")
query_menu.pack(pady=5)

ttk.Button(root, text="Run Query", command=run_query).pack(pady=5)

frame = ttk.Frame(root)
frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

tree = ttk.Treeview(frame)
tree.pack(fill=tk.BOTH, expand=True)

vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
vsb.pack(side='right', fill='y')
hsb.pack(side='bottom', fill='x')

root.mainloop()

C:\Users\Azwa Arshad\AppData\Local\Temp\ipykernel_18236\1545449434.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
